In [1]:
ord('a')

97

In [2]:
chr(97)

'a'

In [ ]:
import sympy
print(sympy.invert(11,26))

from Crypto.Util.number import inverse
print(inverse(11,26))

import gmpy2
print(gmpy2.invert(11,26))

19
19
19


In [5]:
# carser cipher

def caesar_cipher(text, shift):
    result = ""
    for char in text:
        # 小寫字母
        if char.islower():
            result += chr((ord(char) - 97 + shift) % 26 + 97)
        # 大寫字母
        elif char.isupper():
            result += chr( (ord(char) - 65 + shift) % 26 + 65)
        # 非字母字符保持不变
        else:
            result += char
    return result

def caesar_decipher(text, shift):
    return caesar_cipher(text, -shift)

# usecase
encrypted_text = caesar_cipher("Hello World!", 3)
print("Encrypted:", encrypted_text)

decrypted_text = caesar_decipher(encrypted_text, 3)
print("Decrypted:", decrypted_text)

Encrypted: Khoor Zruog!
Decrypted: Hello World!


In [6]:
# affine cipher

import sympy
original = 'abcdefghijklmnopqrstuvwxyz_'
encrypted = 'ifpmluglesecdlqp_rclfrseljpkq'

def main():
    result = ''
    a = sympy.invert(4,27)
    for e in encrypted:
        result += original[((original.index(e)-15) * a) % 27]
    print(result)

if __name__ == '__main__':
    main()

flag_is_every_haxor_love_math


In [7]:
# Rail Fence cipher

FENCES = 1
CIPHERTEXT = "AaY--rpyfneJBeaaX0n-,ZZcs-uXeeSVJ-sh2tioaZ}slrg,-ciE-anfGt.-eCIyss-TzprttFliora{GcouhQIadctm0ltt-FYluuezTyorZ-"

n = len(CIPHERTEXT)

while True:
    FENCES += 1
    Mat = [[] for _ in range(FENCES)]
    Msg = [''] * n
    j = 0
    d = +1
    
    for i in range(n):
        Mat[j].append(i)
        j += d
        if j == 0 or j == FENCES - 1:
            d = -d

    l = 0
    for row in Mat:
        for i in row:
            Msg[i] = CIPHERTEXT[l]
            l += 1

    m = "".join(Msg)
    if 'SharifCTF{' in m:
        break

print("Number of fences:", FENCES)
print("Plaintext:", m)

Number of fences: 21
Plaintext: A-fence-is-a-structure-that-encloses-an-area,-SharifCTF{QmFzZTY0IGlzIGEgZ2VuZXJpYyB0ZXJt},-typically-outdoors.


In [8]:
# Vigenère cipher

from re import match
from hashlib import md5 as md5_
from itertools import product

# MD5 hash function
md5 = lambda x: md5_(x.encode()).hexdigest()

TABLE = "ABCDEFGHIJKLMNOPQRSTUVWXYZ{}"
ENCRYPT, DECRYPT = 1, -1

def vigenere(string, key, mode=ENCRYPT, table=TABLE):
    L = len(key)
    key = [table.index(i) for i in key]
    string = [table.index(i) for i in string]
    cipher = [(v + mode * key[i % L]) % len(table) for i, v in enumerate(string)]
    return ''.join(table[i] for i in cipher)

# 已知密文與MD5雜湊值
key = ".......?????"  # 表示前面已知、後面未知的密鑰
plaintext = "SECCON{...................................}"
ciphertext = "LMIG}RPEDOEEWKJIQIWKJWMNDTSR}TFVUFWYOCBAJBQ"
md5_hash = "f528a6ab914c1ecf856a1d93103948fe"

# 已知前面部分的 key
known_key = ''
for idx, char in enumerate('SECCON{'):
    for k in TABLE:
        if vigenere(char, k) == ciphertext[idx]:
            known_key += k
            break

print(f'known_key = {known_key}')

# 暴力破解剩下的 key
for poss in product(TABLE, repeat=len(key) - len(known_key)):
    try_key = known_key + ''.join(poss)
    decrypted = vigenere(ciphertext, try_key, DECRYPT)
    
    # 驗證解密結果是否符合格式與MD5雜湊
    if match(r'SECCON\{[A-Z]{35}\}', decrypted) and md5(decrypted) == md5_hash:
        print(f'key = {try_key}')
        print(f'plaintext = {decrypted}')
        break


known_key = VIGENER
key = VIGENERECODE
plaintext = SECCON{ABABABCDEDEFGHIJJKLMNOPQRSTTUVWXYYZ}


In [10]:
# one-time pad

import os

def str_to_bin(s):
    return ''.join(format(ord(c), '08b') for c in s)

def bin_to_str(b):
    chars = [chr(int(b[i:i+8], 2)) for i in range(0, len(b), 8)]
    return ''.join(chars)

def xor_binary(b1, b2):
    return ''.join('1' if x != y else '0' for x, y in zip(b1, b2))

def generate_key(length):
    return ''.join(format(x, '08b') for x in os.urandom(length))

# 主流程
plaintext = 'hello123'
plaintext_bin = str_to_bin(plaintext)
key = generate_key(len(plaintext))
cipher_bin = xor_binary(plaintext_bin, key)

print(f'Plaintext: {plaintext}')
print(f'Plaintext Binary: {plaintext_bin}')
print(f'Key:       {key}')
print(f'Cipher Bin:{cipher_bin}')

# 解密
decrypted_bin = xor_binary(cipher_bin, key)
decrypted_text = bin_to_str(decrypted_bin)
print(f'Decrypted Text: {decrypted_text}')

Plaintext: hello123
Plaintext Binary: 0110100001100101011011000110110001101111001100010011001000110011
Key:       0010110010000000101001001111100000101111010110100111001110010001
Cipher Bin:0100010011100101110010001001010001000000011010110100000110100010
Decrypted Text: hello123


In [11]:
# aes

import base64
from Crypto.Cipher import AES
key = base64.b64decode("6v3T yEgjUcQRnWuIhjdTBA==")
ciphertext = base64.b64decode("rW4q3swEuIOEy8RTIp/DCMdNPtdYopSRXKSLYnX9NQe8z+LMsZ6Mx/x8pwGwofdZ")
crypter = AES.new(key, AES.MODE_ECB)
plaintext = crypter .decrypt(ciphertext).decode("utf-8")
print(plaintext)

flag{do_not_let_machines_win_983e8a2d}__________
